In [6]:

import pandas as pd
data = pd.read_csv('/kaggle/input/stock-data/stock_data.csv')
data = data[['Date','AMZN']]
data['Date'] = pd.to_datetime(data['Date'])
data.set_index('Date', inplace=True)
data


/tmp/ipykernel_33/1370990263.py:4: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  data['Date'] = pd.to_datetime(data['Date'])


,AMZN
Date,
2014-06-06 00:00:00-04:00,-1.422434
2014-06-09 00:00:00-04:00,-1.424503
2014-06-10 00:00:00-04:00,-1.419822
2014-06-11 00:00:00-04:00,-1.417162
2014-06-12 00:00:00-04:00,-1.426018
...,...
2024-05-30 00:00:00-04:00,1.682151
2024-05-31 00:00:00-04:00,1.627242
2024-06-03 00:00:00-04:00,1.663466


In [8]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import r2_score
from keras.models import Sequential
from keras.layers import LSTM, Dense



# data = pd.read_csv('/kaggle/input/stock-data/stock_data.csv')


# data = data[['Date', 'Adj Close']]


# data['Date'] = pd.to_datetime(data['Date'])
# data.set_index('Date', inplace=True)


time_steps = 8

# Create sequences of data for training
sequences = []
targets = []
for i in range(len(data) - time_steps):
    seq = data['AMZN'].values[i:i+time_steps]
    label = data['AMZN'].values[i+time_steps]
    sequences.append(seq)
    targets.append(label)

sequences = np.array(sequences)
targets = np.array(targets)

scaler = MinMaxScaler(feature_range=(0, 1))

sequences = scaler.fit_transform(sequences.reshape(-1, 1)).reshape(-1, time_steps, 1)

targets = targets.reshape(-1, 1)
targets = scaler.fit_transform(targets)

split_index = int(0.8 * len(sequences))
X_train, X_test = sequences[:split_index], sequences[split_index:]
y_train, y_test = targets[:split_index], targets[split_index:]

model = Sequential()
model.add(LSTM(units=50, input_shape=(time_steps, 1)))
model.add(Dense(units=1))
model.compile(optimizer='adam', loss='mean_squared_error')

model.fit(X_train, y_train, epochs=10, batch_size=16, validation_data=(X_test, y_test))


predictions = model.predict(X_test)

predictions = scaler.inverse_transform(predictions.reshape(-1, 1))
y_test = scaler.inverse_transform(y_test)


r2 = r2_score(y_test, predictions)
print(f'R^2 Score on Test Data: {r2}')


Epoch 1/10


/opt/conda/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


126/126 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - loss: 0.0794 - val_loss: 9.3712e-04
Epoch 2/10
126/126 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 5.6277e-04 - val_loss: 8.3508e-04
Epoch 3/10
126/126 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 4.2473e-04 - val_loss: 8.1947e-04
Epoch 4/10
126/126 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 4.4903e-04 - val_loss: 8.3199e-04
Epoch 5/10
126/126 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 4.1870e-04 - val_loss: 8.6996e-04
Epoch 6/10
126/126 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 4.2847e-04 - val_loss: 8.3028e-04
Epoch 7/10
126/126 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 4.5015e-04 - val_loss: 9.0338e-04
Epoch 8/10
126/126 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 4.0749e-04 - val_loss: 8.2101e-04
Epoch 9/10
126/126 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 4.3574e-04 - val_loss: 8.2112e-04
Epoch 10/10
126/126 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 3.7857e-04 - val_loss: 8.1220e-04
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
R^2 Score on Test Data: 0.96974408